In [8]:
import warnings
import os
import tensorflow as tf
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
warnings.filterwarnings("ignore", category=UserWarning, module='keras.src.trainers.data_adapters.py_dataset_adapter')

import keras
import numpy as np

from keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator # type: ignore
from keras.models import Model # type: ignore
from keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, Dropout # type: ignore
from keras.optimizers import Adam # type: ignore
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau # type: ignore

from sklearn.utils.class_weight import compute_class_weight

In [9]:
dataset_path = 'Dataset'

In [10]:
train_val_datagen = ImageDataGenerator(
    rescale=1.0 / 255.0,   
    validation_split=0.2 
)

test_datagen = ImageDataGenerator(rescale=1.0 / 255.0)

train_generator = train_val_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='binary',    
    subset='training',     
    shuffle=True
)

validation_generator = train_val_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),  
    batch_size=32,
    class_mode='binary',    
    subset='validation',   
    shuffle=True
)

Found 4686 images belonging to 2 classes.
Found 1170 images belonging to 2 classes.


In [11]:
def build_cnn():
    input_layer = Input(shape=(224, 224, 3))

    x = Conv2D(32, (3, 3), activation='relu', padding='same')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = MaxPooling2D((2, 2))(x)

    x = Flatten()(x)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(256, activation='relu')(x)
    x = Dropout(0.5)(x)
    
    output_layer = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output_layer)

    return model

In [12]:
class_indices = train_generator.class_indices
class_labels = list(class_indices.keys())

class_counts = np.bincount(train_generator.classes)

class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_generator.classes), y=train_generator.classes)
class_weights_dict = dict(enumerate(class_weights))  

In [ ]:
model = build_cnn()

initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True
)

model.compile(
    optimizer=Adam(learning_rate=lr_schedule),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

checkpoint = ModelCheckpoint(
    '/models/best_model.keras', 
    monitor='val_loss', 
    save_best_only=True, 
    mode='min'
)
early_stopping = EarlyStopping(
    monitor='val_loss', 
    patience=10, 
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss', 
    factor=0.2, 
    patience=5, 
    min_lr=0.0001
)


In [14]:
history = model.fit(
    train_generator,
    epochs=100,
    validation_data=validation_generator,
    class_weight=class_weights_dict,
    callbacks=[checkpoint, early_stopping]
)

Epoch 1/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 41s 263ms/step - accuracy: 0.7359 - loss: 0.5360 - val_accuracy: 0.9051 - val_loss: 0.2319
Epoch 2/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 30s 201ms/step - accuracy: 0.9272 - loss: 0.1973 - val_accuracy: 0.8752 - val_loss: 0.2835
Epoch 3/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 34s 233ms/step - accuracy: 0.9309 - loss: 0.1691 - val_accuracy: 0.9350 - val_loss: 0.1747
Epoch 4/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 29s 193ms/step - accuracy: 0.9523 - loss: 0.1218 - val_accuracy: 0.9197 - val_loss: 0.2390
Epoch 5/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 34s 230ms/step - accuracy: 0.9496 - loss: 0.1280 - val_accuracy: 0.9299 - val_loss: 0.1873
Epoch 6/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 35s 239ms/step - accuracy: 0.9559 - loss: 0.1140 - val_accuracy: 0.9342 - val_loss: 0.1898
Epoch 7/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 33s 223ms/step - accuracy: 0.9617 - loss: 0.1052 - val_accuracy: 0.9402 - val_loss: 0.2068
Epoch 8/100
147/147 ━━━━━━━━━━━━━━━━━━━━ 35s 234ms/step - accuracy: 0.9648 -